# Library

In [1]:
import numpy as np
import torch
import gpytorch
import argparse
import time
import pickle
import scipy.io as sio

from torch.distributions import Normal
import matplotlib.pyplot as plt
import random

import roslib
import rospy
import tf as tf_ros
from nav_msgs.msg import Odometry, Path
from sensor_msgs.msg import Image
from cv_bridge import CvBridge
from geometry_msgs.msg import PoseStamped, PoseArray, Pose
import math
import cv2
import copy

import sys
sys.path.append('..')

the rosdep view is empty: call 'sudo rosdep init' and 'rosdep update'


## Check GPU

In [2]:
import torch
import sys
sys.path.append('..')
from torchlib.utils import list_device,set_device

list_device()

------------ List Devices ------------
Device 0 :
GeForce RTX 2060
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB

Device 1 :
TITAN Xp
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB



## Set torch default parameters

In [3]:
set_device(1)
torch.set_default_dtype(torch.float32)
torch.set_printoptions(precision=4)
torch.backends.cudnn.benchmark = True
torch.set_printoptions(sci_mode=False)

Using Device 1 : TITAN Xp


# Set Arguments

In [4]:
import argparse
import sys
import os
import time
import pickle

parser = argparse.ArgumentParser()
parser.add_argument('--batch_size', type=int, default=300, help='size of mini batch')
parser.add_argument('--is_normalization', type=bool, default=True, help='whether do data normalization')
parser.add_argument('--target_image_size', default=[300, 300], nargs=2, type=int, help='Input images will be resized to this for data argumentation.')
parser.add_argument('--output_dim', default=3, type=int, help='output dimention.')
parser.add_argument('--feat_dim', default=128, type=int, help='feature dimention.')
parser.add_argument('--model_dir', type=str, default='/notebooks/global_localization/gps_net_torch', help='rnn, gru, or lstm')

parser.add_argument('--test_dataset', type=str, default=[# '/notebooks/michigan_nn_data/2012_01_08',
                                                         # '/notebooks/michigan_nn_data/2012_01_15',
                                                         # '/notebooks/michigan_nn_data/2012_01_22',
                                                         # '/notebooks/michigan_nn_data/2012_02_02',
                                                         # '/notebooks/michigan_nn_data/2012_02_04',
                                                         # '/notebooks/michigan_nn_data/2012_02_05',
                                                         '/notebooks/michigan_nn_data/2012_02_12',
                                                         # '/notebooks/michigan_nn_data/2012_03_31',
                                                         '/notebooks/michigan_nn_data/2012_04_29',
                                                         '/notebooks/michigan_nn_data/2012_05_11',
                                                         '/notebooks/michigan_nn_data/2012_06_15',
                                                         '/notebooks/michigan_nn_data/2012_08_04',
                                                         # '/notebooks/michigan_nn_data/2012_09_28'])
                                                         '/notebooks/michigan_nn_data/2012_10_28',
                                                         '/notebooks/michigan_nn_data/2012_11_16',
                                                         '/notebooks/michigan_nn_data/2012_12_01'
                                                        ] )

parser.add_argument('--train_dataset', type=str, default = ['/notebooks/michigan_nn_data/test'])
parser.add_argument('--norm_tensor', type=str, default = ['/notebooks/global_localization/norm_mean_std.pt'])

#parser.add_argument('--map_dataset', type=str, default='/home/kevin/data/michigan_gt/training')
parser.add_argument('--enable_ros', type=bool, default=False, help='put data into ros')

sys.argv = ['']
args = parser.parse_args()

if args.enable_ros:
    rospy.init_node('global_localization_tf_broadcaster_cnn')

# Load Dataset

In [5]:
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import tf.transformations as tf_tran
from tqdm import tqdm
#from PIL import Image
import numpy as np
import random

import torch.nn as nn
import torch.optim as optim
from torchlib import resnet, vggnet, cnn_auxiliary
from torchlib.cnn_auxiliary import normalize, denormalize, denormalize_navie, get_relative_pose, translational_rotational_loss
from torchlib.utils import LocalizationDataset, display_loss, data2tensorboard
import time

transform = transforms.Compose([transforms.ToTensor()])
dataset = LocalizationDataset(dataset_dirs = args.test_dataset, \
                              image_size = args.target_image_size, \
                              transform = transform,
                              get_pair = False, mode='evaluate')
if len(args.train_dataset)>7:
    [args.norm_mean, args.norm_std] = [torch.tensor(x) for x in dataset.get_norm()]
    torch.save([args.norm_mean, args.norm_std], *args.norm_tensor)
    print('Save norm and std:',*args.norm_tensor)
else:
    [args.norm_mean, args.norm_std] = torch.load(*args.norm_tensor)
    print('Load norm and std:',*args.norm_tensor)

dataloader = DataLoader(dataset, batch_size=args.batch_size, \
                        shuffle=False, num_workers=0, \
                        drop_last=False, pin_memory=True)

100%|██████████| 12683/12683 [00:16<00:00, 777.29it/s]


Load norm and std: /notebooks/global_localization/norm_mean_std.pt


# Define Model

In [6]:
class Backbone(nn.Module):
    def __init__(self):
        super().__init__()
        self.resnet = resnet.resnet50(pretrained=True)
    def forward(self,input_data):
        dense_feat = self.resnet(input_data)
        return dense_feat
    
class NN(nn.Module):
    def __init__(self):
        super().__init__()
        self.global_context = vggnet.vggnet(input_channel=2048,opt="context")
        self.global_regressor = vggnet.vggnet(opt="regressor")
        
    def forward(self,input_data):
        context_feat = self.global_context(input_data)
        output,feature_t, feature_r = self.global_regressor(context_feat)
        return output, feature_t, feature_r

class GP(gpytorch.models.ApproximateGP):
    def __init__(self, inducing_points, output_dim=3):
        variational_distribution = gpytorch.variational.CholeskyVariationalDistribution(
            inducing_points.size(-2), batch_shape=torch.Size([output_dim])
        )
        variational_strategy = gpytorch.variational.MultitaskVariationalStrategy(
            gpytorch.variational.VariationalStrategy(
                self, inducing_points, variational_distribution, learn_inducing_locations=True
            ), num_tasks=output_dim
        )
        super().__init__(variational_strategy)
        self.mean_module = gpytorch.means.ConstantMean(batch_shape=torch.Size([1]))
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.RBFKernel(batch_shape=torch.Size([1])),
            batch_shape=torch.Size([1]))

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

class GPNode(nn.Module):
    def __init__(self,inducing_points, seed):
        super().__init__()
        output_dim = inducing_points.shape[0]
        sub_feat_dim = inducing_points.shape[-1]
        torch.manual_seed(seed)
        
        self.feat_index = torch.randint(high=args.feat_dim, size=(sub_feat_dim,))
        self.gp = GP(inducing_points)
        self.likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=output_dim) 
        
    def forward(self,input_data):
        output = self.gp(input_data)
        return output
    
class Model(nn.Module):
    def __init__(self,gp_args):
        super().__init__()
        self.backbone = Backbone()
        self.nn = NN()
        self.gps = nn.ModuleList()
        
        self.num_gp = gp_args['num_gp']
        self.sub_batch_rate = gp_args['sub_rate']
        self.sub_feat_rate = gp_args['feat_rate']
        self.sub_batch_size = int(args.batch_size*self.sub_batch_rate)
        self.sub_feat_dim = int(args.feat_dim*self.sub_feat_rate)
        
        for i in range(self.num_gp):
            inducing_points = torch.zeros(args.output_dim, self.sub_batch_size, self.sub_feat_dim)
            # use i as seed to fix sub features
            gp = GPNode(inducing_points,seed=i)
            self.gps.append(gp)
        
    def forward_nn(self, input_data):
        dense_feat = self.backbone(input_data)
        output, feature_t, feature_r = self.nn(dense_feat)
        rot_pred = torch.split(output, [3, 4], dim=1)[1] # 4-dimention            
        return feature_t, rot_pred
    
    def forward_gp(self,gp,trans_feat):
        sub_trans_feat = trans_feat[:,gp.feat_index]
        trans_pred = gp(sub_trans_feat)
        return trans_pred

In [7]:
class Trainer:
    def __init__(self,gp_args,regressor_context_rate = [0.0,0.0]):
        self.model = Model(gp_args).cuda()
        self.norm_mean = args.norm_mean.cuda()
        self.norm_std = args.norm_std.cuda()
        
        # disable learning backbone
        for param in self.model.backbone.parameters():
            param.requires_grad = False
        '''
        # training tool
        self.optimizer = optim.Adam(self._optimize(regressor_context_rate))
        self.scheduler = optim.lr_scheduler.LambdaLR(optimizer=self.optimizer,
                                                         lr_lambda=lambda epoch: args.decay_rate**epoch)
        '''
    def load_model(self, file_name = 'pretrained.pth'):
        # load file info
        state_dict = torch.load(os.path.join(args.model_dir, file_name))
        if 'net.resnet.conv1.weight' in state_dict:
            print('Transform from old model.')
            # Part 1: backbone
            backbone_state_dict = self._from_old_model(state_dict,'backbone')
            print('Backbone parameters layer:',len(backbone_state_dict.keys()))
            self.model.backbone.load_state_dict(backbone_state_dict,strict = True)
            # Part 2: nn
            nn_state_dict = self._from_old_model(torch.load(os.path.join(args.model_dir, file_name)),'nn')
            print('NN parameters layer:',len(nn_state_dict.keys()))
            self.model.nn.load_state_dict(nn_state_dict,strict = True)
        else:
            print('Parameters layer:',len(state_dict.keys()))
            # load file to model
            self.model.load_state_dict(state_dict,strict = True)
        print('Model Structure:')
        # Display model structure
        for name, param in self.model.named_parameters():
            print(name, param.shape)
        print('Parameters layer:',len(self.model.state_dict().keys()))
    
    def _from_old_model(self, state_dict, select = 'backbone'):
        if select == 'backbone':
            for key in list(state_dict):
                if 'net.resnet.' in key:
                    state_dict[key.replace('net.resnet.','resnet.')] = state_dict.pop(key)
                else:
                    state_dict.pop(key)
        elif select == 'nn':
            for key in list(state_dict):
                if 'net.global_regressor.' in key:
                    state_dict[key.replace('net.global_regressor.','global_regressor.')] = state_dict.pop(key)
                elif 'net.global_context.' in key:
                    state_dict[key.replace('net.global_context.','global_context.')] = state_dict.pop(key)
                else:
                    state_dict.pop(key)
        return state_dict
    
    def save_model(self, file_name = 'model-{}-{}.pth'):
        checkpoint_path = os.path.join(args.model_dir, file_name)
        torch.save(self.model.state_dict(),checkpoint_path)
        print('Saving model to ' +  file_name)
        
    def _optimize(self,regressor_context_rate = [0.0,0.0]):
        optimizer = [
                {'params': self.model.gps.parameters(), \
                 'lr': args.learning_rate,'weight_decay':args.weight_decay}]
            
        if regressor_context_rate[0]!=0:
            optimizer += [{'params': self.model.nn.global_regressor.parameters(), \
                 'lr': args.learning_rate * regressor_context_rate[0],'weight_decay':args.weight_decay}]
            print('Regressor learn rate:',regressor_context_rate[0])
        else:
            for param in self.model.nn.global_regressor.parameters():
                param.requires_grad = False
                
        if regressor_context_rate[1]!=0:
            optimizer += [{'params': self.model.nn.global_context.parameters(), \
                 'lr': args.learning_rate * regressor_context_rate[1],'weight_decay':args.weight_decay}]
            print('Context learn rate:',regressor_context_rate[1])
        else:
            for param in self.model.nn.global_context.parameters():
                param.requires_grad = False
                
        return optimizer
            
    def train(self,x,y):
        # Step 0: zero grad
        self.optimizer.zero_grad()
        
        start = time.time()
        # Step 1: get data
        x,y = x.cuda(),y.cuda()
        if args.is_normalization:
            y = normalize(y,self.norm_mean, self.norm_std)
            
        # Step 2: training
        assert trainer.model.training == True
        
        trans_loss = torch.tensor(0.).cuda()
        
        trans_target, rot_target = torch.split(y, [3, 4], dim=1)
        trans_feat, rot_pred = self.model.forward_nn(x)
        rot_loss = self._nn_loss(rot_pred,rot_target)
        for gp in self.model.gps:
            sampled_mask = torch.randint(high=args.batch_size, size=(self.model.sub_batch_size,))
            sub_x = trans_feat[sampled_mask]
            sub_y = trans_target[sampled_mask]
            gp_loss = self._gp_loss(gp,sub_x,sub_y)
            trans_loss += gp_loss
        trans_loss = trans_loss/self.model.num_gp
        
        total_loss = trans_loss + args.lamda_weights * rot_loss
        
        batch_time = time.time() - start
        
        #Step 3: update
        total_loss.backward()
        self.optimizer.step()
        
        return float(total_loss), batch_time    
    
    def _nn_loss(self,rot_pred,rot_target):
        rot_loss = 1. - torch.mean(torch.square(torch.sum(torch.mul(rot_pred,rot_target),dim=1)))
        return rot_loss
        
    def _gp_loss(self,gp,trans_feat,trans_target):
        # predict
        trans_pred = self.model.forward_gp(gp,trans_feat)
        #sub_trans_feat = trans_feat[:,gp.feat_index]
        #trans_pred = gp(sub_trans_feat)
        
        num_data = int(min(len(dataloader)*args.batch_size,len(dataset))*self.model.sub_batch_rate)
        mll = gpytorch.mlls.PredictiveLogLikelihood(gp.likelihood, gp.gp, num_data = num_data)
        
        # trans loss
        trans_loss = -1.*mll(trans_pred, trans_target)
        
        return trans_loss
    
    def _eval_gp(self, gp, trans_pred):
        c_mean, c_var = trans_pred.mean, trans_pred.variance
        y_mean, y_var = gp.likelihood(trans_pred).mean, gp.likelihood(trans_pred).variance
        
        return y_mean, c_mean, c_var
    
    def _sample(self, mean, var, num_sample = 100):
        dist = Normal(mean, var)
        samples = dist.sample([num_sample])
        return samples

    def eval_forward(self,x,y,num_sample = 100,output_denormalize = True):
        # Step 1: get data
        x,y = x.cuda(),y.cuda()
        if args.is_normalization:
            y = normalize(y,self.norm_mean, self.norm_std)
        
        # Step 2: forward
        assert trainer.model.training == False
        trans_feat, rot_pred = self.model.forward_nn(x)
        
        trans_preds = 0
        trans_means = 0
        trans_vars = 0
        for gp in self.model.gps:
            trans_pred = self.model.forward_gp(gp,trans_feat)
            trans_pred, trans_mean, trans_var = self._eval_gp(gp, trans_pred)
            trans_preds += trans_pred
            trans_means += trans_mean
            trans_vars += trans_var
            
        trans_preds /= self.model.num_gp
        trans_means /= self.model.num_gp
        trans_vars /= self.model.num_gp
        
        if args.is_normalization and output_denormalize:
            trans_preds = denormalize_navie(trans_preds, self.norm_mean, self.norm_std)
            trans_means = denormalize_navie(trans_means, self.norm_mean, self.norm_std)
            trans_vars = trans_vars.mul(self.norm_std)
            y = denormalize(y, self.norm_mean, self.norm_std)
        
        samples = self._sample(trans_means, trans_vars, num_sample)
            
        # Step 3: split output
        trans_target, rot_target = torch.split(y, [3, 4], dim=1)
        
        return trans_preds, rot_pred, trans_target, rot_target, samples
    
gp_args = {
    'feat_rate':0.85,
    'num_gp':3,
    'sub_rate':0.7
}    

trainer = Trainer(gp_args,regressor_context_rate = [0.,0.])
trainer.load_model('pretrained.pth')

Parameters layer: 376
Model Structure:
backbone.resnet.conv1.weight torch.Size([64, 1, 7, 7])
backbone.resnet.bn1.weight torch.Size([64])
backbone.resnet.bn1.bias torch.Size([64])
backbone.resnet.layer1.0.conv1.weight torch.Size([64, 64, 1, 1])
backbone.resnet.layer1.0.bn1.weight torch.Size([64])
backbone.resnet.layer1.0.bn1.bias torch.Size([64])
backbone.resnet.layer1.0.conv2.weight torch.Size([64, 64, 3, 3])
backbone.resnet.layer1.0.bn2.weight torch.Size([64])
backbone.resnet.layer1.0.bn2.bias torch.Size([64])
backbone.resnet.layer1.0.conv3.weight torch.Size([256, 64, 1, 1])
backbone.resnet.layer1.0.bn3.weight torch.Size([256])
backbone.resnet.layer1.0.bn3.bias torch.Size([256])
backbone.resnet.layer1.0.downsample.0.weight torch.Size([256, 64, 1, 1])
backbone.resnet.layer1.0.downsample.1.weight torch.Size([256])
backbone.resnet.layer1.0.downsample.1.bias torch.Size([256])
backbone.resnet.layer1.1.conv1.weight torch.Size([64, 256, 1, 1])
backbone.resnet.layer1.1.bn1.weight torch.Size(

In [8]:
for param in trainer.model.parameters():
    param.requires_grad = False

# Initialize

In [9]:
trans_errors = []
rot_errors = []
uncertainties = []
pose_map = []

total_trans_error = 0.
total_rot_error = 0.

count = 0.

is_save_map = False
is_read_map = False

trans_preds = []
trans_gts = []

rot_preds = []
rot_gts = []

pred_uncertainties = []

pred_time = []

br = tf_ros.TransformBroadcaster()

GT_POSE_TOPIC = '/gt_pose'
BIRDVIEW_TOPIC_PUB = '/bird_view'
MAP_TOPIC_PUB = '/pose_map'
PARTICLES_PUB = '/particles'
NN_LOCALIZASION_PUB = '/nn_pose'
gt_pose_pub = rospy.Publisher(GT_POSE_TOPIC, Odometry, queue_size=1)
bird_view_pub = rospy.Publisher(BIRDVIEW_TOPIC_PUB, Image, queue_size=1)
map_pub = rospy.Publisher(MAP_TOPIC_PUB, Path, queue_size=1)
particles_pub = rospy.Publisher(PARTICLES_PUB, PoseArray, queue_size=1)
nn_pose_pub = rospy.Publisher(NN_LOCALIZASION_PUB, Odometry, queue_size=1)

In [10]:
trainer.model.eval()

for b, data in enumerate(dataloader, 0):
    start = time.time()
    x,y = data.values()
    trans_pred, rot_pred, trans_gt, rot_gt, samples = trainer.eval_forward(x,y)
    
    # transform data
    trans_pred = trans_pred.cpu().numpy()
    rot_pred = rot_pred.cpu().numpy()
    trans_gt = trans_gt.cpu().numpy()
    rot_gt = rot_gt.cpu().numpy()
    
    end = time.time()
    
    if args.enable_ros:
        particles = PoseArray()
        particles.header.stamp = rospy.Time.now()
        particles.header.frame_id = 'world'
        for s in samples:
            pose = Pose()
            [pose.position.x, pose.position.y, pose.position.z] = s
            [pose.orientation.x, pose.orientation.y, pose.orientation.z, pose.orientation.w] = rot_pred[0]
            particles.poses.append(pose)
        particles_pub.publish(particles)

        [px_pred, py_pred, pz_pred] = trans_pred[0]
        [qx_pred, qy_pred, qz_pred, qw_pred] = rot_pred[0]

        br.sendTransform((px_pred, py_pred, pz_pred),
                         (qx_pred, qy_pred, qz_pred, qw_pred), rospy.Time.now(),
                         "estimation", "world")

        [px_gt, py_gt, pz_gt] = trans_gt[0]
        [qx_gt, qy_gt, qz_gt, qw_gt] = rot_gt[0]

        br.sendTransform((px_gt, py_gt, pz_gt),
                         (qx_gt, qy_gt, qz_gt, qw_gt),
                         rospy.Time.now(), "gt", "world")

        timestamp = rospy.Time.now()

        nn_pose_msg = Odometry()
        nn_pose_msg.header.frame_id = 'world'
        nn_pose_msg.header.stamp = timestamp
        nn_pose_msg.child_frame_id = 'base_link'
        nn_pose_msg.pose.pose.position.x = px_pred
        nn_pose_msg.pose.pose.position.y = py_pred
        nn_pose_msg.pose.pose.position.z = pz_pred
        [nn_pose_msg.pose.pose.orientation.x, nn_pose_msg.pose.pose.orientation.y, nn_pose_msg.pose.pose.orientation.z, nn_pose_msg.pose.pose.orientation.w] = [qx_pred, qy_pred, qz_pred, qw_pred]

        conv = np.zeros((6,6), dtype=np.float32)
        [conv[0][0], conv[1][1], conv[2][2]] = trans_cov[0]
        nn_pose_msg.pose.covariance = conv.flatten().tolist()
        nn_pose_pub.publish(nn_pose_msg)

        bridge = CvBridge()

        bird_view_img_msg = bridge.cv2_to_imgmsg(np.asarray(x[0].cpu(), dtype=np.float32), encoding="passthrough")
        stamp_now = rospy.Time.now()
        bird_view_img_msg.header.stamp = stamp_now

        bird_view_pub.publish(bird_view_img_msg)

        rospy.sleep(.0)
        cv2.waitKey(0)

        count += 1
    else:
        count += y.shape[0]
    
    trans_preds += [x for x in trans_pred]
    rot_preds += [x for x in rot_pred]
    trans_gts += [x for x in trans_gt]
    rot_gts += [x for x in rot_gt]

    trans_error = np.sqrt(np.sum((trans_pred - trans_gt)**2,axis=1))
    rot_error_1 = np.arccos(np.sum(np.multiply(rot_pred,rot_gt),axis=1))/math.pi*180
    rot_error_2 = np.arccos(np.sum(np.multiply(rot_pred,-rot_gt),axis=1))/math.pi*180
    rot_error = np.minimum(rot_error_1,rot_error_2)

    trans_errors += [x for x in trans_error]
    rot_errors += [x for x in rot_error]

    total_trans_error += np.sum(trans_error)
    total_rot_error += np.sum(rot_error)
    
    display = 1

    if b % display == 0:
        print(
            "{}/{}, translation error = {:.3f}, rotation error = {:.3f}, time/batch = {:.3f}"
            .format(
             (b+1)*args.batch_size,
            len(dataloader)*args.batch_size,
            total_trans_error / count,
            total_rot_error / count,
            end - start))

300/92100, translation error = 3.964, rotation error = 2.797, time/batch = 4.024
600/92100, translation error = 9.535, rotation error = 3.462, time/batch = 0.782
900/92100, translation error = 9.448, rotation error = 3.404, time/batch = 0.776
1200/92100, translation error = 8.014, rotation error = 3.459, time/batch = 0.778
1500/92100, translation error = 7.034, rotation error = 3.583, time/batch = 0.773
1800/92100, translation error = 6.161, rotation error = 3.610, time/batch = 0.774
2100/92100, translation error = 5.825, rotation error = 3.732, time/batch = 0.774
2400/92100, translation error = 5.379, rotation error = 3.631, time/batch = 0.773
2700/92100, translation error = 5.137, rotation error = 3.747, time/batch = 0.785
3000/92100, translation error = 4.988, rotation error = 3.758, time/batch = 0.778
3300/92100, translation error = 4.701, rotation error = 3.637, time/batch = 0.777
3600/92100, translation error = 4.608, rotation error = 3.659, time/batch = 0.776
3900/92100, transla

30300/92100, translation error = 8.761, rotation error = 5.055, time/batch = 0.793
30600/92100, translation error = 8.700, rotation error = 5.041, time/batch = 0.787
30900/92100, translation error = 8.637, rotation error = 5.032, time/batch = 0.788
31200/92100, translation error = 8.584, rotation error = 5.041, time/batch = 0.780
31500/92100, translation error = 8.529, rotation error = 5.022, time/batch = 0.783
31800/92100, translation error = 8.473, rotation error = 5.001, time/batch = 0.788
32100/92100, translation error = 8.423, rotation error = 4.986, time/batch = 0.784
32400/92100, translation error = 8.369, rotation error = 4.979, time/batch = 0.788
32700/92100, translation error = 8.312, rotation error = 4.970, time/batch = 0.783
33000/92100, translation error = 8.250, rotation error = 4.954, time/batch = 0.782
33300/92100, translation error = 8.203, rotation error = 4.937, time/batch = 0.794
33600/92100, translation error = 8.160, rotation error = 4.927, time/batch = 0.783
3390

60000/92100, translation error = 9.997, rotation error = 5.281, time/batch = 0.793
60300/92100, translation error = 9.960, rotation error = 5.272, time/batch = 0.795
60600/92100, translation error = 9.928, rotation error = 5.268, time/batch = 0.790
60900/92100, translation error = 9.892, rotation error = 5.260, time/batch = 0.789
61200/92100, translation error = 9.868, rotation error = 5.256, time/batch = 0.789
61500/92100, translation error = 9.911, rotation error = 5.278, time/batch = 0.788
61800/92100, translation error = 10.158, rotation error = 5.412, time/batch = 0.788
62100/92100, translation error = 10.119, rotation error = 5.408, time/batch = 0.788
62400/92100, translation error = 10.086, rotation error = 5.404, time/batch = 0.789
62700/92100, translation error = 10.049, rotation error = 5.397, time/batch = 0.792
63000/92100, translation error = 10.015, rotation error = 5.394, time/batch = 0.786
63300/92100, translation error = 10.014, rotation error = 5.409, time/batch = 0.78

89400/92100, translation error = 11.696, rotation error = 6.585, time/batch = 0.788
89700/92100, translation error = 11.668, rotation error = 6.577, time/batch = 0.789
90000/92100, translation error = 11.725, rotation error = 6.582, time/batch = 0.788
90300/92100, translation error = 11.692, rotation error = 6.570, time/batch = 0.791
90600/92100, translation error = 11.671, rotation error = 6.567, time/batch = 0.792
90900/92100, translation error = 11.642, rotation error = 6.561, time/batch = 0.791
91200/92100, translation error = 11.624, rotation error = 6.592, time/batch = 0.792
91500/92100, translation error = 11.600, rotation error = 6.670, time/batch = 0.789
91800/92100, translation error = 11.569, rotation error = 6.660, time/batch = 0.792
92100/92100, translation error = 11.558, rotation error = 6.660, time/batch = 2.538


In [11]:
sio.savemat('results.mat', {'trans_pred': np.array(trans_preds), 'trans_gt': np.array(trans_gts), 'uncertainty': np.array(pred_uncertainties)})

if len(pose_map):
    np.savetxt(os.path.join(args.map_dataset, 'map.txt'), np.asarray(pose_map, dtype=np.float32))
    print("map is saved!")

plt.hist(trans_errors, bins='auto')
plt.title("Translation errors")
plt.xlabel("translational error in meters")
plt.ylabel("number of frames")
plt.savefig('terror.png', bbox_inches='tight')

plt.hist(rot_errors, bins='auto')
plt.title("Rotation errors")
plt.xlabel("rotational error in degree")
plt.ylabel("number of frames")
plt.savefig('rerror.png', bbox_inches='tight')

median_trans_errors = np.median(trans_errors)
median_rot_errors = np.median(rot_errors)
mean_trans_errors = np.mean(trans_errors)
mean_rot_errors = np.mean(rot_errors)

print("median translation error = {:.3f}".format(median_trans_errors))
print("median rotation error = {:.3f}".format(median_rot_errors))
print("mean translation error = {:.3f}".format(mean_trans_errors))
print("mean rotation error = {:.3f}".format(mean_rot_errors))   

median translation error = 2.495
median rotation error = 3.161
mean translation error = 11.558
mean rotation error = 6.660


In [12]:
def evaluate(trans_errors,rot_errors):
    t = [14301,7008,12852,9567,13580,14835,7114,12683]
    for i in range(len(t)):
        if i >0:
            t[i] += t[i-1]
    trans_errors_month = list()
    trans_errors_month.append(trans_errors[:t[0]])
    trans_errors_month.append(trans_errors[t[0]:t[1]])
    trans_errors_month.append(trans_errors[t[1]:t[2]])
    trans_errors_month.append(trans_errors[t[2]:t[3]])
    trans_errors_month.append(trans_errors[t[3]:t[4]])
    trans_errors_month.append(trans_errors[t[4]:t[5]])
    trans_errors_month.append(trans_errors[t[5]:t[6]])
    trans_errors_month.append(trans_errors[t[6]:])

    rot_errors_month = list()
    rot_errors_month.append(rot_errors[:t[0]])
    rot_errors_month.append(rot_errors[t[0]:t[1]])
    rot_errors_month.append(rot_errors[t[1]:t[2]])
    rot_errors_month.append(rot_errors[t[2]:t[3]])
    rot_errors_month.append(rot_errors[t[3]:t[4]])
    rot_errors_month.append(rot_errors[t[4]:t[5]])
    rot_errors_month.append(rot_errors[t[5]:t[6]])
    rot_errors_month.append(rot_errors[t[6]:])
    
    print('================== median translation error ==================')
    for trans_errors_i in trans_errors_month:
        print("median translation error = {:.3f}".format(np.median(trans_errors_i)))
        
    print('================== median rotation error ==================')
    for rot_errors_i in rot_errors_month:
        print("median rotation error = {:.3f}".format(np.median(rot_errors_i)))
    
    print('================== mean translation error ==================')
    for trans_errors_i in trans_errors_month:
        print("mean translation error = {:.3f}".format(np.mean(trans_errors_i)))
        
    print('================== mean rotation error ==================')  
    for rot_errors_i in rot_errors_month:
        print("mean rotation error = {:.3f}".format(np.mean(rot_errors_i)))
        
evaluate(trans_errors,rot_errors)

================== median translation error ==================
median translation error = 2.070
median translation error = 2.017
median translation error = 2.373
median translation error = 2.421
median translation error = 2.509
median translation error = 2.467
median translation error = 4.085
median translation error = 3.264
================== median rotation error ==================
median rotation error = 2.603
median rotation error = 2.829
median rotation error = 3.002
median rotation error = 2.957
median rotation error = 3.193
median rotation error = 3.321
median rotation error = 4.465
median rotation error = 4.189
================== mean translation error ==================
mean translation error = 4.747
mean translation error = 4.879
mean translation error = 13.542
mean translation error = 13.820
mean translation error = 9.777
mean translation error = 11.914
mean translation error = 22.905
mean translation error = 14.338
================== mean rotation error ==================
m